In [ ]:
from pandas import read_csv
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df_top10s = pd.read_csv("../input/top-spotify-songs-from-20102019-by-year/top10s.csv", engine='python') # the engine needs to be changed otherwise UTF-8 error occurs
df_top10s.head()

In [ ]:
df_top10s.info()

In [ ]:
df_top10s.info()

In [ ]:
df_top10s['Count']=1
df_genre=df_top10s.groupby('top genre')['Count'].sum().reset_index().sort_values(by='Count',ascending=False)
df_genre.head()

In [ ]:
 len(df_top10s["top genre"].unique())

In [ ]:
df_top10s["top genre"].value_counts()

In [ ]:
sns.catplot(y = "top genre", kind = "count",
            palette = "bright", edgecolor = ".8",
            data = df_top10s)
plt.show()

In [ ]:
# first extracting the genre columns
# getting rid of white spaces and turning it all into lower cases
genre = (df_top10s["top genre"].str.strip()).str.lower()
genre.head()

In [ ]:
def genre_splitter(genre):
    result = genre.copy()
    result = result.str.split(" ",1)
    for i in range(len(result)):
        if (len(result[i]) > 1):
            result[i] = [result[i][1]]
    return result.str.join('')
genre_splitter(genre)

In [ ]:
# loop until the genre cannot be split any further
genre_m1 = genre.copy()
while(max((genre_m1.str.split(" ", 1)).str.len()) > 1):
    genre_m1 = genre_splitter(genre_m1)

In [ ]:
len(genre_m1.unique())

In [ ]:
genre_m1.value_counts()

In [ ]:
unique = genre_m1.unique()
to_remove = [] 

# genres that have a single instance only will be placed within the to_remove array
for genre in unique:
    if genre_m1.value_counts()[genre] < 20: # 10 was arbitrarily chosen
        to_remove += [genre]
len(to_remove)

In [ ]:
df_top10s['Top Genre'] = genre_m1
df_top10s

In [ ]:
df_top10s.set_index(["Top Genre"],drop = False, inplace = True)
for name in to_remove:
    type(name)
    df_top10s.drop(index = str(name), inplace = True)

In [ ]:
df_top10s["Top Genre"].value_counts()

In [ ]:
train_set, test_set = train_test_split(df_top10s, test_size = 0.2, random_state = 42)
# training set
X_train = train_set.values[:,1:]
y_train = train_set.values[:,0]

# test set
X_test = test_set.values[:,1:]
y_test = test_set.values[:,0]

In [ ]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler().fit(X_train)

# Standard Scaler
X_train_ST = standard_scaler.transform(X_train)
X_test_ST = standard_scaler.transform(X_test)

In [ ]:
attributes = df_top10s.columns[1:]
for attribute in attributes:
    temp = df_top10s[attribute]
    for instance in range(len(temp)):
        if(type(temp[instance]) == str):
            df_top10s[attribute][instance] = float(temp[instance].replace(',',''))
# check data types using df.dtype